<a href="https://colab.research.google.com/github/AashiDutt/AI-and-ML-for-Coders/blob/main/Chapter_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This colab contains code for:
1. Tokenization and Padding
2. IMDb dataset exploration
3. Getting text from CSV files (Twitter Sentiment Analysis- Binary emotion Dataset)
4. Getting text from JSON Files (Sarcasm Dataset)

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer 

In [ ]:
sentences= [
            'Today is a sunny day',
            'Today is a rainy day',
            'Is it sunny today?', #tokenizer removes punctuation
            'I really enjoyed walking in the snow today'
            
]

tokenizer = Tokenizer(num_words = 100) #specify no. of words tokenizer can tokenize
tokenizer.fit_on_texts(sentences) #creates tokenized word index
word_index = tokenizer.word_index
print(word_index)

{'today': 1, 'is': 2, 'a': 3, 'sunny': 4, 'day': 5, 'rainy': 6, 'it': 7, 'i': 8, 'really': 9, 'enjoyed': 10, 'walking': 11, 'in': 12, 'the': 13, 'snow': 14}


In [ ]:
#converting sentences in list of numbers
# numbers - value
# word - key

sequences = tokenizer.texts_to_sequences(sentences)

print(sequences)

[[1, 2, 3, 4, 5], [1, 2, 3, 6, 5], [2, 7, 4, 1], [8, 9, 10, 11, 12, 13, 14, 1]]


"Today is a sunny day" is encoded as  - [1, 2, 3, 4, 5]

"Today is a rainy day" is encoded as - [1, 2, 3, 6, 5]

"Is it sunny today?" is encoded as - [2, 7, 4, 1]

In [ ]:
#for previously unseen data

test_data = [
             'Today is a snowy day',
             'Will it be rainy tomorrow?'
]

test_sequences = tokenizer.texts_to_sequences(test_data)
print(word_index)
print(test_sequences)

{'today': 1, 'is': 2, 'a': 3, 'sunny': 4, 'day': 5, 'rainy': 6, 'it': 7, 'i': 8, 'really': 9, 'enjoyed': 10, 'walking': 11, 'in': 12, 'the': 13, 'snow': 14}
[[1, 2, 3, 5], [7, 6]]


Swapping back tokens 

[7,6] - it rainy (Actual - will it be rainy tomorrow?)

In [ ]:
#Out of vocabulary token 
tokenizer = Tokenizer(num_words = 100, oov_token = "<OOV>")
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(sentences)

test_sequences = tokenizer.texts_to_sequences(test_data)
print(word_index)
print(test_sequences)

{'<OOV>': 1, 'today': 2, 'is': 3, 'a': 4, 'sunny': 5, 'day': 6, 'rainy': 7, 'it': 8, 'i': 9, 'really': 10, 'enjoyed': 11, 'walking': 12, 'in': 13, 'the': 14, 'snow': 15}
[[2, 3, 4, 1, 6], [1, 8, 1, 7, 1]]


Reverse encoding now will give 

[2,3,4,1,6] - today is a <OOV> day

[1,8,1,7,1] - <OOV> it <OOV> rainy <OOV> 

Most of words of 2nd sentence were not in corpus, thus it lacks context but it's a step in the right direction.

In [ ]:
from os import truncate
#Different sentences = different sizes(length)
#Padding helps to get all sentences in same shape and size

#from tensorflow.keras.preprocessing.sequence
#import pad_sequences
#padded = pad_sequences(sequences)

padded = tf.keras.preprocessing.sequence.pad_sequences(sequences,padding = 'post', truncating='post')
print(padded)


[[ 2  3  4  5  6  0  0  0]
 [ 2  3  4  7  6  0  0  0]
 [ 3  8  5  2  0  0  0  0]
 [ 9 10 11 12 13 14 15  2]]


IMDb Dataset

In [ ]:
import tensorflow_datasets as tfds

In [ ]:
#removing stopwords that are too common and repeated
from bs4 import BeautifulSoup
import string 

stopwords = ["a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at",
             "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do",
             "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having",
             "he", "hed", "hes", "her", "here", "heres", "hers", "herself", "him", "himself", "his", "how",
             "hows", "i", "id", "ill", "im", "ive", "if", "in", "into", "is", "it", "its", "itself",
             "lets", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought",
             "our", "ours", "ourselves", "out", "over", "own", "same", "she", "shed", "shell", "shes", "should",
             "so", "some", "such", "than", "that", "thats", "the", "their", "theirs", "them", "themselves", "then",
             "there", "theres", "these", "they", "theyd", "theyll", "theyre", "theyve", "this", "those", "through",
             "to", "too", "under", "until", "up", "very", "was", "we", "wed", "well", "were", "weve", "were",
             "what", "whats", "when", "whens", "where", "wheres", "which", "while", "who", "whos", "whom", "why",
             "whys", "with", "would", "you", "youd", "youll", "youre", "youve", "your", "yours", "yourself",
             "yourselves"]

#removing Punctuations -otherwise, a stopword followed by comma won't be spotted
table = str.maketrans('','',string.punctuation)

#Get the Data
imdb_sentences = []
train_data = tfds.as_numpy(tfds.load('imdb_reviews', split = 'train'))

for item in train_data:
  sentence = str(item['text'].decode('UTF-8').lower()) #loading data as strings

#adding space around punctuation (him/her becomes him / her) and can be removed easily.

  sentence = sentence.replace(",", " , ") 
  sentence = sentence.replace(".", " . ")
  sentence = sentence.replace("-", " - ")
  sentence = sentence.replace("/", " / ")
  soup = BeautifulSoup(sentence)
  sentence = soup.get_text()
  words = sentence.split()
  filtered_sentence = ""
  for word in words:
    word = word.translate(table)
    if word not in stopwords:
      filtered_sentence = filtered_sentence + word + " "
  imdb_sentences.append(filtered_sentence)
  

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete308LCT/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete308LCT/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
#Assigning Tokens
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = 25000)
tokenizer.fit_on_texts(imdb_sentences)
sequences = tokenizer.texts_to_sequences(imdb_sentences)
print(tokenizer.word_index)

{'movie': 1, 'film': 2, 'not': 3, 'one': 4, 'like': 5, 'just': 6, 'good': 7, 'no': 8, 'time': 9, 'even': 10, 'story': 11, 'really': 12, 'see': 13, 'can': 14, 'much': 15, 'bad': 16, 'get': 17, 'will': 18, 'also': 19, 'people': 20, 'great': 21, 'first': 22, 'dont': 23, 'made': 24, 'movies': 25, 'make': 26, 'way': 27, 'films': 28, 'characters': 29, 'think': 30, 'watch': 31, 'two': 32, 'character': 33, 'many': 34, 'seen': 35, 'life': 36, 'plot': 37, 'never': 38, 'acting': 39, 'little': 40, 'love': 41, 'best': 42, 'show': 43, 'know': 44, 'off': 45, 'ever': 46, 'better': 47, 'end': 48, 'still': 49, 'man': 50, 'say': 51, 'scene': 52, 'scenes': 53, 'go': 54, 'something': 55, 'back': 56, 'real': 57, 'watching': 58, 'years': 59, 'now': 60, 'doesnt': 61, 'though': 62, 'actors': 63, 'old': 64, 'thing': 65, 'didnt': 66, 'work': 67, 'new': 68, 'another': 69, 'nothing': 70, 'funny': 71, 'actually': 72, '10': 73, 'makes': 74, 'director': 75, 'look': 76, 'find': 77, 'going': 78, 'part': 79, 'every': 80

In [ ]:
#Reverse encoding - sequence to sentence

reverse_word_index = dict(
    [(value,key) for (key,value) in
     tokenizer.word_index.items()])
decoded_review = ' '.join([reverse_word_index.get(i, '?')
for i in sequences[0]])

print(decoded_review)

absolutely terrible movie dont lured christopher walken michael ironside great actors must simply worst role history even great acting not redeem movies ridiculous storyline movie early nineties us propaganda piece pathetic scenes rebels making cases revolutions maria conchita alonso appeared phony pseudo love affair walken nothing pathetic emotional plug movie devoid real meaning disappointed movies like ruining actors like christopher walkens good name barely sit


IMDB Subwords dataset

In [ ]:
(train_data, test_data), info = tfds.load('imdb_reviews/subwords8k',
                                          split=(tfds.Split.TRAIN, tfds.Split.TEST),
                                          as_supervised = True,
                                          with_info = True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteC7VDSQ/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteC7VDSQ/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


In [ ]:
encoder = info.features['text'].encoder
print('Vocabulary size: {}'.format(encoder.vocab_size))

Vocabulary size: 8185


In [ ]:
print(encoder.subwords)

['the_', ', ', '. ', 'a_', 'and_', 'of_', 'to_', 's_', 'is_', 'br', 'in_', 'I_', 'that_', 'this_', 'it_', ' /><', ' />', 'was_', 'The_', 'as_', 't_', 'with_', 'for_', '.<', 'on_', 'but_', 'movie_', ' (', 'are_', 'his_', 'have_', 'film_', 'not_', 'ing_', 'be_', 'ed_', 'you_', ' "', 'it', 'd_', 'an_', 'he_', 'by_', 'at_', 'one_', 'who_', 'y_', 'from_', 'e_', 'or_', 'all_', 'like_', 'they_', '" ', 'so_', 'just_', 'has_', ') ', 'her_', 'about_', 'out_', 'This_', 'some_', 'ly_', 'movie', 'film', 'very_', 'more_', 'It_', 'would_', 'what_', 'when_', 'which_', 'good_', 'if_', 'up_', 'only_', 'even_', 'their_', 'had_', 'really_', 'my_', 'can_', 'no_', 'were_', 'see_', 'she_', '? ', 'than_', '! ', 'there_', 'get_', 'been_', 'into_', ' - ', 'will_', 'much_', 'story_', 'because_', 'ing', 'time_', 'n_', 'we_', 'ed', 'me_', ': ', 'most_', 'other_', 'don', 'do_', 'm_', 'es_', 'how_', 'also_', 'make_', 'its_', 'could_', 'first_', 'any_', "' ", 'people_', 'great_', 've_', 'ly', 'er_', 'made_', 'r_', 'B

In [ ]:
sample_string = "Today is a sunny day"
encoded_string = encoder.encode(sample_string)
print('Encoded string is {}'.format(encoded_string))

Encoded string is [6427, 4869, 9, 4, 2365, 1361, 606]


In [ ]:
print(encoder.subwords[6426]) #addressing an array

Tod


In [ ]:
#decoding

orignal_string = encoder.decode(encoded_string)
test_string = encoder.decode([6427, 4869, 9, 4, 2365, 1361, 606])

print(test_string)

Today is a sunny day


Getting Text from CSV files

Twitter Sentiment Analysis- Binary emotion Dataset

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
!wget --no-check-certificate --no-cache \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/binary-emotion.csv \
    -O /tmp/binary-emotion.csv

--2022-02-03 04:04:52--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/binary-emotion.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.203.128, 172.253.123.128, 142.250.98.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2690504 (2.6M) [text/csv]
Saving to: ‘/tmp/binary-emotion.csv’

/tmp/binary-emotion 100%[===================>]   2.57M  --.-KB/s    in 0.02s   

2022-02-03 04:04:53 (126 MB/s) - ‘/tmp/binary-emotion.csv’ saved [2690504/2690504]



In [2]:
import string

stopwords = ["a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at",
             "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do",
             "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having",
             "he", "hed", "hes", "her", "here", "heres", "hers", "herself", "him", "himself", "his", "how",
             "hows", "i", "id", "ill", "im", "ive", "if", "in", "into", "is", "it", "its", "itself",
             "lets", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought",
             "our", "ours", "ourselves", "out", "over", "own", "same", "she", "shed", "shell", "shes", "should",
             "so", "some", "such", "than", "that", "thats", "the", "their", "theirs", "them", "themselves", "then",
             "there", "theres", "these", "they", "theyd", "theyll", "theyre", "theyve", "this", "those", "through",
             "to", "too", "under", "until", "up", "very", "was", "we", "wed", "well", "were", "weve", "were",
             "what", "whats", "when", "whens", "where", "wheres", "which", "while", "who", "whos", "whom", "why",
             "whys", "with", "would", "you", "youd", "youll", "youre", "youve", "your", "yours", "yourself",
             "yourselves"]

table = str.maketrans('', '', string.punctuation)

In [4]:
from bs4 import BeautifulSoup

In [5]:
import csv

sentences = []
labels = []

with open('/tmp/binary-emotion.csv', encoding = 'UTF-8') as csvfile:
  reader = csv.reader(csvfile, delimiter = ",")

  for row in reader:
    labels.append(int(row[0]))
    sentence = row[1].lower()
    sentence = sentence.replace(","," , ")
    sentence = sentence.replace("."," . ")
    sentence = sentence.replace("-"," - ")
    sentence = sentence.replace("/"," / ")
    soup = BeautifulSoup(sentence)
    sentence = soup.get_text()
    words = sentence.split()
    filtered_sentence = ""
    
    for word in words:
      word = word.translate(table)
      if word not in stopwords:
        filtered_sentence = filtered_sentence + word + ""
      sentences.append(filtered_sentence)

In [6]:
#creating training and test subsets
# total sentences = 35,327

training_size = 28000

training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

In [9]:
#creating word index from training set
vocab_size = 20000
max_length = 10 # max len of sentence
trunc_type = 'post' # truncate rest of sentence
padding_type = 'post'
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen = max_length, padding = padding_type, truncating = trunc_type)

print(training_sequences[1])
print(training_padded[1])
print(word_index)

[13]
[13  0  0  0  0  0  0  0  0  0]
{'<OOV>': 1, 'just': 2, 'wish': 3, 'oh': 4, 'dont': 5, 'cant': 6, 'not': 7, 'yeah': 8, 'want': 9, 'hate': 10, 'need': 11, 'think': 12, 'know': 13, 'sorry': 14, 'no': 15, 'cantsleep': 16, 'aww': 17, 'going': 18, 'love': 19, 'com': 20, 'miss': 21, 'ok': 22, 'omg': 23, 'ugh': 24, 'feeling': 25, 'haha': 26, 'got': 27, 'feel': 28, 'morning': 29, 'trying': 30, 'tired': 31, 'sad': 32, 'now': 33, 'back': 34, 'justgot': 35, 'lol': 36, 'can': 37, 'still': 38, 'wow': 39, 'time': 40, 'dontthink': 41, 'come': 42, 'really': 43, 'wishes': 44, 'thought': 45, 'wondering': 46, 'dude': 47, 'damn': 48, 'justgothome': 49, '3': 50, 'sucks': 51, 'work': 52, 'off': 53, 'notexcitedblowinganothercandletoday': 54, 'missed': 55, 'aw': 56, 'home': 57, 'didnt': 58, 'went': 59, 'guess': 60, 'good': 61, 'feellike': 62, 'bummed': 63, 'friday': 64, 'gonna': 65, 'sleepy': 66, 'hey': 67, 'ohno': 68, 'go': 69, 'awww': 70, 'wheredsongsgositewant': 71, 'thingsmummysaidneedlearnwashdishes

Getting Text from JSON files

**Sarcasm Dataset**


In [1]:
import tensorflow_datasets as tfds

In [7]:
from bs4 import BeautifulSoup
import tensorflow as tf


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [9]:
import string

stopwords = ["a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at",
             "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do",
             "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having",
             "he", "hed", "hes", "her", "here", "heres", "hers", "herself", "him", "himself", "his", "how",
             "hows", "i", "id", "ill", "im", "ive", "if", "in", "into", "is", "it", "its", "itself",
             "lets", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought",
             "our", "ours", "ourselves", "out", "over", "own", "same", "she", "shed", "shell", "shes", "should",
             "so", "some", "such", "than", "that", "thats", "the", "their", "theirs", "them", "themselves", "then",
             "there", "theres", "these", "they", "theyd", "theyll", "theyre", "theyve", "this", "those", "through",
             "to", "too", "under", "until", "up", "very", "was", "we", "wed", "well", "were", "weve", "were",
             "what", "whats", "when", "whens", "where", "wheres", "which", "while", "who", "whos", "whom", "why",
             "whys", "with", "would", "you", "youd", "youll", "youre", "youve", "your", "yours", "yourself",
             "yourselves"]

table = str.maketrans('', '', string.punctuation)

In [10]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json \
    -O /tmp/sarcasm.json
  
import json
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

with open("/tmp/sarcasm.json", 'r') as f:
    datastore = json.load(f)


sentences = [] 
labels = []
urls = []

for item in datastore:
    sentence = item['headline'].lower()
    sentence = sentence.replace(",", " , ")
    sentence = sentence.replace(".", " . ")
    sentence = sentence.replace("-", " - ")
    sentence = sentence.replace("/", " / ")
    soup = BeautifulSoup(sentence)
    sentence = soup.get_text()
    words = sentence.split()
    filtered_sentence = ""
    for word in words:
        word = word.translate(table)
        if word not in stopwords:
            filtered_sentence = filtered_sentence + word + " "
    sentences.append(filtered_sentence)
    labels.append(item['is_sarcastic'])
    urls.append(item['article_link'])

--2022-02-02 16:01:11--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.199.128, 74.125.20.128, 74.125.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.199.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘/tmp/sarcasm.json’

/tmp/sarcasm.json   100%[===================>]   5.38M  --.-KB/s    in 0.03s   

2022-02-02 16:01:12 (195 MB/s) - ‘/tmp/sarcasm.json’ saved [5643545/5643545]



In [11]:
print(len(sentences))

26709


In [12]:
#total sentences = 26709  (Train-Test split)
training_size = 23000

training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

In [13]:
vocab_size = 20000
max_length = 10
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(training_sequences, padding='post')
print(word_index)

{'<OOV>': 1, 'new': 2, 'trump': 3, 'man': 4, 'not': 5, 'just': 6, 'will': 7, 'one': 8, 'year': 9, 'report': 10, 'area': 11, 'donald': 12, 'day': 13, 'u': 14, 's': 15, 'can': 16, 'says': 17, 'woman': 18, 'first': 19, 'time': 20, 'like': 21, 'no': 22, 'get': 23, 'old': 24, 'trumps': 25, 'off': 26, 'now': 27, 'life': 28, 'obama': 29, 'people': 30, 'house': 31, 'women': 32, 'white': 33, 'back': 34, 'still': 35, 'make': 36, 'clinton': 37, '5': 38, 'world': 39, 'americans': 40, 'years': 41, 'way': 42, 'family': 43, 'black': 44, 'study': 45, 'gop': 46, 'best': 47, 'bill': 48, 'american': 49, 'school': 50, 'police': 51, 'watch': 52, '3': 53, 'show': 54, 'really': 55, 'president': 56, 'home': 57, 'last': 58, 'video': 59, 'finds': 60, 'death': 61, 'cant': 62, 'know': 63, '10': 64, 'things': 65, 'good': 66, 'going': 67, 'state': 68, 'hillary': 69, 'love': 70, 'nation': 71, 'may': 72, '2': 73, 'health': 74, 'say': 75, 'parents': 76, 'big': 77, 'every': 78, 'right': 79, 'work': 80, '000': 81, 'need